# Homegrown Model Building
Creates homegrown models out of various pre-trained resnet models. 

In [ ]:
import csv 
import time
import csv
from PIL import Image
import numpy as np
import scipy.misc
import scipy.ndimage
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from sklearn import preprocessing
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.applications import resnet50

# Config Params

In [ ]:
# Define where all datasets live
data_basepath = '{IMAGE PATH}'

# Define which dataset to use
dataset_name = 'uo_dress_tiny'

# Support Function - Confusion Matrix Plotter

In [ ]:
# custom confusion matrix plotter
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

plt.rcParams['figure.figsize'] = [20, 5]

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# load in the dresses dataset

In [ ]:
image_basepath   = data_basepath+'/'+dataset_name+'/'
feature_basepath = data_basepath+'/'+dataset_name+'/resnet50_features_vectors/'
augmented_image_directory_name = data_basepath+'/'+dataset_name+'/augmented/train/'
augmentded_data_path           = data_basepath+'/'+dataset_name+'/augmented/train/full/'
cropped_image_directory_name = data_basepath+'/'+dataset_name+'/cropped/'
csv_filename = data_basepath+'/'+dataset_name+'/labels.csv'

stime = time.time()

train_filenames = []
val_filenames   = []
test_filenames  = []
y_train = []
y_test  = []
y_val   = []

with open(csv_filename,'r') as f:
    reader = csv.reader(f,delimiter=',')
    for line in reader:
        # print(line)
        #sys.exit()
        
        filename = line[0]
        dataset = line[0].split('/')[0]
        label = line[1]
        
        if dataset == 'test':
            y_test.append(label)
            test_filenames.append(filename)
        elif dataset == 'val':
            y_val.append(label)
            val_filenames.append(filename)
        elif dataset == 'train':
            y_train.append(label)
            train_filenames.append(filename)


In [ ]:
x_train = []
x_val   = []
x_test  = []

id = dict()
id['train'] = dict()
id['test'] = dict()
id['val'] = dict()

N = 224
for i,filename in enumerate(train_filenames):
    pil_im = Image.open(image_basepath+filename,'r')
    im3 = scipy.misc.imresize(np.asarray(pil_im), (N,N,3))
    x_train.append(im3)
    id[filename.split('/')[0]][filename.split('/')[-1]] = i

for i,filename in enumerate(val_filenames):
    pil_im = Image.open(image_basepath+filename,'r')
    im3 = scipy.misc.imresize(np.asarray(pil_im), (N,N,3))
    x_val.append(im3)
    id[filename.split('/')[0]][filename.split('/')[-1]] = i

for i,filename in enumerate(test_filenames):
    pil_im = Image.open(image_basepath+filename,'r')
    im3 = scipy.misc.imresize(np.asarray(pil_im), (N,N,3))
    x_test.append(im3)
    id[filename.split('/')[0]][filename.split('/')[-1]] = i

x_train = np.array(x_train)
x_val = np.array(x_val)
x_test = np.array(x_test)

print(x_train.shape)
print(x_test.shape)
print(x_val.shape)


In [ ]:
le = preprocessing.LabelEncoder()

le.fit(y_train)
print(list(le.classes_))

y_train_num = np.array(le.transform(y_train)) 
y_test_num  = np.array(le.transform(y_test))
y_val_num   = np.array(le.transform(y_val))

y_train_cat = np.array(to_categorical(le.transform(y_train))) 
y_test_cat  = np.array(to_categorical(le.transform(y_test)))
y_val_cat   = np.array(to_categorical(le.transform(y_val)))

print(y_train_cat.shape)
print(y_test_cat.shape)
print(y_val_cat.shape)

# basic CNN

In [ ]:
x_train_cnn = x_train.astype('float32')
x_test_cnn = x_test.astype('float32')
x_val_cnn = x_val.astype('float32')
x_train_cnn /= 255.0
x_test_cnn /= 255.0
x_val_cnn /= 255.0

In [ ]:
batch_size = 128
num_classes = 4
epochs = 200

# input image dimensions
input_shape = (N, N, 3)

print('x_train_cnn shape:', x_train_cnn.shape)
print('x_val_cnn shape:', x_val_cnn.shape)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(8, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

print(model.summary())

In [ ]:
checkpointer = ModelCheckpoint(filepath=dataset_name+'_'+'basic_cnn_weights.hdf5', verbose=0, save_best_only=True)
earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, verbose=0, mode='auto')

model.fit(x_train_cnn, y_train_cat,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_val_cnn, y_val_cat),
         callbacks=[checkpointer,earlystopping])

In [ ]:
model.load_weights(dataset_name+'_'+'basic_cnn_weights.hdf5')


score = model.evaluate(x_test_cnn, y_test_cat, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
import collections
import pickle
import perfreport

y_pred_scores = model.predict(x_test_cnn) 
y_pred = np.argmax(y_pred_scores,axis=1)

print(collections.Counter(le.inverse_transform(y_pred)))

results = {'y_true': np.array(y_test),
           'scores': np.array(y_pred_scores),
           'class_labels': np.array(list(le.classes_)),
           'y_pred': np.array(le.inverse_transform(y_pred))}

pickle.dump( results, open( dataset_name+'_'+'basic_cnn.p', "wb" ) )

In [ ]:
perfreport.print_metrics(dataset_name+'_'+'basic_cnn.p')

# ResNet50 as feature extractor, 2xDNN on top

In [ ]:
# load VGG with pretrained imagenet weights (once with max pooling and once without)
from keras.models import Model
from keras.layers import Dense,Flatten
from keras.applications import resnet50
from keras import backend as K

with open( feature_basepath + "resnet50_full.p", "r" ) as f:
    features = pickle.load(f)
    
print(features.keys())


In [ ]:
x_train_resnet50 = features['x_train']
x_val_resnet50   = features['x_val']
x_test_resnet50  = features['x_test']

y_train_cat = features['y_train_cat']
y_val_cat   = features['y_val_cat']
y_test_cat  = features['y_test_cat']

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 4
epochs = 200

# input image dimensions
input_shape = (2048,)

model = Sequential()
model.add(Dense(32, activation='relu',input_shape=input_shape))
model.add(Dropout(0.25))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

print(model.summary())

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpointer = ModelCheckpoint(filepath=dataset_name+'_'+'resnet50_weights.hdf5', verbose=0, save_best_only=True)
earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=25, verbose=0, mode='auto')

model.fit(x_train_resnet50, y_train_cat,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_val_resnet50, y_val_cat),
         callbacks=[checkpointer])

In [ ]:
from sklearn.metrics import confusion_matrix

model.load_weights(dataset_name+'_'+'resnet50_weights.hdf5')


p = model.predict(x_test_resnet50)
p2 = np.argmax(p,axis=1)
cm = confusion_matrix(y_test_num, p2)

score = model.evaluate(x_test_resnet50, y_test_cat, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print("")
print(cm)

In [ ]:
plt.figure()
plot_confusion_matrix(cm, classes=['1','2','3','4'],
                      title='Confusion matrix, without normalization', normalize=True)
plt.show()

In [ ]:
y_pred_scores = model.predict(x_test_resnet50) 
y_pred = np.argmax(y_pred_scores,axis=1)

import collections
print(collections.Counter(le.inverse_transform(y_pred)))

print(np.array(x_test_resnet50).shape)
print(np.array(y_pred_scores).shape)
print(np.array(list(le.classes_)).shape)
print(np.array(le.inverse_transform(y_pred)).shape)

results = {'y_true': np.array(y_test),
           'scores': np.array(y_pred_scores),
           'class_labels': np.array(list(le.classes_)),
           'y_pred': np.array(le.inverse_transform(y_pred))}

pickle.dump( results, open( dataset_name+'_'+'resnet50_as_feature_extractor.p', "wb" ) )
perfreport.print_metrics(dataset_name+'_'+'resnet50_as_feature_extractor.p')

# Unlock last CNN layer of Resnet and retrain 

In [ ]:
model_resnet50 = resnet50.ResNet50(include_top=False, weights='imagenet')

batch_size = 128
num_classes = 4
epochs = 200

input_shape = (2048,)

for i,layer in enumerate(model_resnet50.layers):
    if i >= 170:
        layer.trainable = True
    else: 
        layer.trainable = False
    print(i,layer.trainable,layer)

model = Sequential()
model.add(model_resnet50)
model.add(Reshape((2048,)))
model.add(Dense(32, activation='relu',input_shape=input_shape))
model.add(Dropout(0.25))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

print(model.summary())


In [ ]:
checkpointer = ModelCheckpoint(filepath=dataset_name+'_'+'resnet50_retrain_last_layer_weights.hdf5', verbose=0, save_best_only=True)
earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=4, verbose=0, mode='auto')

model.fit(x_train, y_train_cat,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_val, y_val_cat),
         callbacks=[checkpointer,earlystopping])

In [ ]:
model.load_weights(dataset_name+'_'+'resnet50_retrain_last_layer_weights.hdf5')

y_pred_scores = model.predict(x_test) 
y_pred = np.argmax(y_pred_scores,axis=1)

import collections
print(collections.Counter(le.inverse_transform(y_pred)))

print(np.array(x_test).shape)
print(np.array(y_pred_scores).shape)
print(np.array(list(le.classes_)).shape)
print(np.array(le.inverse_transform(y_pred)).shape)

results = {'y_true': np.array(y_test),
           'scores': np.array(y_pred_scores),
           'class_labels': np.array(list(le.classes_)),
           'y_pred': np.array(le.inverse_transform(y_pred))}

pickle.dump( results, open( dataset_name+'_'+'resnet50_retrain_last_CNN_layer.p', "wb" ) )
perfreport.print_metrics(dataset_name+'_'+'resnet50_retrain_last_CNN_layer.p')

# Unlock last CNN layer of Resnet and retrain on augmented data

In [ ]:
import glob
import time

x_train_aug = []
y_train_aug = []

stime = time.time()

filenames = glob.glob(augmentded_data_path+"*.*")
for i,filename in enumerate(filenames):
    if i % 10000 == 0:
        print(i,time.time()-stime)
    pil_im = Image.open(filename,'r')
    im3 = scipy.misc.imresize(np.asarray(pil_im), (N,N,3))
    x_train_aug.append(im3)
    
    filename_end = filename.split('/')[-1]
    filename_end = '_'.join(filename_end.split('_')[2:-2])+'.jpg'
    
    #index = int(filename.split('/')[-1].split('_')[0])
    index = id['train'][filename_end]
    y_train_aug.append(y_train_cat[index,:])

x_train_aug = np.array(x_train_aug)
y_train_aug = np.array(y_train_aug)
print(x_train_aug.shape)
print(y_train_aug.shape)


In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 4
epochs = 200

input_shape = (2048,)

for i,layer in enumerate(model_resnet50.layers):
    if i >= 170:
        layer.trainable = True
    else: 
        layer.trainable = False
    #print(i,layer.trainable,layer)

model = Sequential()
model.add(model_resnet50)
model.add(Reshape((2048,)))
model.add(Dense(32, activation='relu',input_shape=input_shape))
model.add(Dropout(0.25))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

print(model.summary())

In [ ]:
checkpointer = ModelCheckpoint(filepath=dataset_name+'_'+'resnet50_retrain_last_layer_augmented_weights.hdf5', verbose=0, save_best_only=True)
earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=4, verbose=0, mode='auto')

model.fit(x_train_aug, y_train_aug,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_data=(x_val, y_val_cat),
         callbacks = [checkpointer,earlystopping])

In [ ]:
model.load_weights(dataset_name+'_'+'resnet50_retrain_last_layer_augmented_weights.hdf5')

y_pred_scores = model.predict(x_test) 
y_pred = np.argmax(y_pred_scores,axis=1)

import collections
print(collections.Counter(le.inverse_transform(y_pred)))

print(np.array(x_test).shape)
print(np.array(y_pred_scores).shape)
print(np.array(list(le.classes_)).shape)
print(np.array(le.inverse_transform(y_pred)).shape)

results = {'y_true': np.array(y_test),
           'scores': np.array(y_pred_scores),
           'class_labels': np.array(list(le.classes_)),
           'y_pred': np.array(le.inverse_transform(y_pred))}

pickle.dump( results, open( dataset_name+'_'+'resnet50_retrain_last_CNN_layer_with_augmentation.p', "wb" ) )
perfreport.print_metrics(dataset_name+'_'+'resnet50_retrain_last_CNN_layer_with_augmentation.p')

# resnet50 as feature extractor on augmented data

In [ ]:
# load VGG with pretrained imagenet weights (once with max pooling and once without)
from keras.models import Model
from keras.layers import Dense,Flatten
from keras.applications import resnet50
from keras import backend as K

with open( feature_basepath + "resnet50_full_augmented.p", "r" ) as f:
    augmented_features = pickle.load(f)

with open( feature_basepath + "resnet50_full.p", "r" ) as f:
    features = pickle.load(f)

print(augmented_features.keys())
print(features.keys())


In [ ]:
x_train_resnet50 = augmented_features['x_train']
x_val_resnet50   = features['x_val']
x_test_resnet50  = features['x_test']

y_train_cat = augmented_features['y_train_cat']
y_val_cat   = features['y_val_cat']
y_test_cat  = features['y_test_cat']

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 4
epochs = 200

# input image dimensions
input_shape = (2048,)

model = Sequential()
model.add(Dense(32, activation='relu',input_shape=input_shape))
model.add(Dropout(0.25))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

print(model.summary())

In [ ]:
checkpointer = ModelCheckpoint(filepath=dataset_name+'_'+'resnet50_augmented_weights.hdf5', verbose=0, save_best_only=True)
earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=20, verbose=0, mode='auto')


model.fit(x_train_resnet50, y_train_aug,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_val_resnet50, y_val_cat),
        callbacks = [checkpointer,earlystopping])

In [ ]:
model.load_weights(dataset_name+'_'+'resnet50_augmented_weights.hdf5')

y_pred_scores = model.predict(x_test_resnet50) 
y_pred = np.argmax(y_pred_scores,axis=1)

import collections
print(collections.Counter(le.inverse_transform(y_pred)))

print(np.array(x_test_resnet50).shape)
print(np.array(y_pred_scores).shape)
print(np.array(list(le.classes_)).shape)
print(np.array(le.inverse_transform(y_pred)).shape)

results = {'y_true': np.array(y_test),
           'scores': np.array(y_pred_scores),
           'class_labels': np.array(list(le.classes_)),
           'y_pred': np.array(le.inverse_transform(y_pred))}

pickle.dump( results, open( dataset_name+'_'+'resnet50_as_feature_extractor_augmented_data.p', "wb" ) )
perfreport.print_metrics(dataset_name+'_'+'resnet50_as_feature_extractor_augmented_data.p')

# ensemble of augmented and raw resnet

In [ ]:
with open( dataset_name+'_'+'resnet50_as_feature_extractor_augmented_data.p', "r" ) as f:
    raw = pickle.load(f)
print(raw.keys())

with open( dataset_name+'_'+'resnet50_as_feature_extractor.p', "r" ) as f:
    aug = pickle.load(f)
print(aug.keys())

y_pred_combined = np.argmax(aug['scores']+raw['scores'],axis=1)
print(y_pred_combined.shape)

aug['scores'] = (aug['scores']+raw['scores'])/2.0
aug['y_pred'] = np.array(le.inverse_transform(y_pred_combined))

pickle.dump( aug, open( dataset_name+'_'+'resnet50_as_feature_extractor_augmented_and_raw_data.p', "wb" ) )
perfreport.print_metrics(dataset_name+'_'+'resnet50_as_feature_extractor_augmented_and_raw_data.p')

# resnet as feature extractor using both uncropped and zoomed images

In [ ]:
# load VGG with pretrained imagenet weights (once with max pooling and once without)
from keras.models import Model
from keras.layers import Dense,Flatten
from keras.applications import resnet50
from keras import backend as K

with open( feature_basepath + "resnet50_cropped.p", "r" ) as f:
    cropped_features = pickle.load(f)

with open( feature_basepath + "resnet50_full.p", "r" ) as f:
    features = pickle.load(f)

print(cropped_features.keys())
print(features.keys())

In [ ]:
x_train_resnet50_cropped = cropped_features['x_train']
x_val_resnet50_cropped   = cropped_features['x_val']
x_test_resnet50_cropped  = cropped_features['x_test']

x_train_resnet50 = features['x_train']
x_val_resnet50   = features['x_val']
x_test_resnet50  = features['x_test']

y_train_cropped_cat = cropped_features['y_train_cat']
y_val_cropped_cat   = cropped_features['y_val_cat']
y_test_cropped_cat  = cropped_features['y_test_cat']

y_train_cat = features['y_train_cat']
y_val_cat   = features['y_val_cat']
y_test_cat  = features['y_test_cat']

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.layers import Merge

batch_size = 128
num_classes = 4
epochs = 200

# input image dimensions
input_shape = (2048,)

model_raw = Sequential()
model_raw.add(Dense(32, activation='relu',input_shape=input_shape))
model_raw.add(Dropout(0.25))
model_raw.add(Dense(32, activation='relu'))
model_raw.add(Dropout(0.25))
model_raw.add(Dense(num_classes, activation='softmax'))

model_crop = Sequential()
model_crop.add(Dense(32, activation='relu',input_shape=input_shape))
model_crop.add(Dropout(0.25))
model_crop.add(Dense(32, activation='relu'))
model_crop.add(Dropout(0.25))
model_crop.add(Dense(num_classes, activation='softmax'))

model = Sequential()
model.add(Merge([model_raw, model_crop], mode = 'concat', concat_axis=1))
model.add(Dense(32, activation='relu',kernel_initializer='random_uniform'))
model.add(Dense(32, activation='relu',kernel_initializer='random_uniform'))
model.add(Dense(num_classes, activation='softmax',kernel_initializer='random_uniform'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

print(model.summary())

In [ ]:
checkpointer = ModelCheckpoint(filepath=dataset_name+'_'+'resnet50_augmented_cropped_and_raw_weights.hdf5', verbose=0, save_best_only=True)
earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=50, verbose=0, mode='auto')


model.fit([x_train_resnet50,x_train_resnet50_cropped], y_train_cat,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([x_val_resnet50,x_val_resnet50_cropped], y_val_cat),
        callbacks = [checkpointer,earlystopping])

In [ ]:
model.load_weights(dataset_name+'_'+'resnet50_augmented_cropped_and_raw_weights.hdf5')

y_pred_scores = model.predict([x_test_resnet50,x_test_resnet50_cropped]) 
y_pred = np.argmax(y_pred_scores,axis=1)

import collections
print(collections.Counter(le.inverse_transform(y_pred)))

print(np.array(x_test_resnet50).shape)
print(np.array(y_pred_scores).shape)
print(np.array(list(le.classes_)).shape)
print(np.array(le.inverse_transform(y_pred)).shape)

results = {'y_true': np.array(y_test),
           'scores': np.array(y_pred_scores),
           'class_labels': np.array(list(le.classes_)),
           'y_pred': np.array(le.inverse_transform(y_pred))}

pickle.dump( results, open( dataset_name+'_'+'resnet50_as_feature_extractor_cropped_and_raw.p', "wb" ) )
perfreport.print_metrics(dataset_name+'_'+'resnet50_as_feature_extractor_cropped_and_raw.p')